In [1]:
#importing libraries
import pandas as pd
from dfply import *
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import os
%matplotlib inline

In [2]:
#setting display options for pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
#coding ideas and comments from https://stackoverflow.com/questions/42900757/sequentially-read-huge-csv-file-in-python/42960918
#trying to understand what would be the best way to handle these data if they were huge
file = '/Users/ariafredman/Documents/data_science/data/star_wars_info_data/characters.csv'
#checking out the first 5 rows of the data
#not necessary for these small data
#but could be good if our data are super large
print(pd.read_csv(file, nrows = 5))

             name  height  mass hair_color   skin_color eye_color birth_year  \
0  Luke Skywalker     172    77      blond         fair      blue      19BBY   
1           C-3PO     167    75        NaN         gold    yellow     112BBY   
2           R2-D2      96    32        NaN  white, blue       red      33BBY   
3     Darth Vader     202   136       none        white    yellow    41.9BBY   
4     Leia Organa     150    49      brown        light     brown      19BBY   

   gender homeworld species  
0    male  Tatooine   Human  
1     NaN  Tatooine   Droid  
2     NaN     Naboo   Droid  
3    male  Tatooine   Human  
4  female  Alderaan   Human  


In [4]:
#creating the sqllite database
csv_database = create_engine('sqlite:///csv_database.db')

In [5]:
#iterating through the CSV file in chunks and store the data into sqllite
#setting the chunksize at 100,000 to keep the size of the chunks managable
#The for loop read a chunk of data from the CSV file, 
#If you want to remove space from any of column names
#use df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
#then stores the chunk into the sqllite database (df.to_sql(…))
chunksize = 100000
i = 0
j = 1
for df in pd.read_csv(file, chunksize = chunksize, iterator = True):
      df.index += j
      i+=1
      df.to_sql('table', csv_database, if_exists = 'append')
      j = df.index[-1] + 1

In [6]:
#we can now pull the data from the query
df1 = pd.read_sql_query("SELECT * FROM 'table' LIMIT 5", csv_database)
print(df1)

   index            name  height mass hair_color   skin_color eye_color  \
0      1  Luke Skywalker   172.0   77      blond         fair      blue   
1      2           C-3PO   167.0   75       None         gold    yellow   
2      3           R2-D2    96.0   32       None  white, blue       red   
3      4     Darth Vader   202.0  136       none        white    yellow   
4      5     Leia Organa   150.0   49      brown        light     brown   

  birth_year  gender homeworld species  
0      19BBY    male  Tatooine   Human  
1     112BBY    None  Tatooine   Droid  
2      33BBY    None     Naboo   Droid  
3    41.9BBY    male  Tatooine   Human  
4      19BBY  female  Alderaan   Human  


In [7]:
df_char_sql = pd.read_sql_query("SELECT * FROM 'table'", csv_database)

In [8]:
#another option if you can pull it all into memory
#but maybe the reading in part is the problematic one
#this data output isn't dataframe, but pandas.io.parsers.TextFileReader
it_df_char = pd.read_csv('/Users/ariafredman/Documents/data_science/data/star_wars_info_data/characters.csv', iterator = True, chunksize=1000)
print(it_df_char)

In [9]:
#If you need a dataframe use concat for all chunks to df
df_char_it = pd.concat(it_df_char, ignore_index = True)
df_char_it.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species
0,Luke Skywalker,172.0,77,blond,fair,blue,19BBY,male,Tatooine,Human
1,C-3PO,167.0,75,NaN,gold,yellow,112BBY,NaN,Tatooine,Droid
2,R2-D2,96.0,32,NaN,"white, blue",red,33BBY,NaN,Naboo,Droid
3,Darth Vader,202.0,136,none,white,yellow,41.9BBY,male,Tatooine,Human
4,Leia Organa,150.0,49,brown,light,brown,19BBY,female,Alderaan,Human


In [10]:
#reading the data in the usual way
df_char_usual = pd.read_csv('/Users/ariafredman/Documents/data_science/data/star_wars_info_data/characters.csv')

In [11]:
#Checking that all df_char_usual and df_char_it are the same
df_char_usual.equals(df_char_it)

True

In [12]:
#so these aren't identical
#so let's look into that a bit more
df_char_usual.equals(df_char_sql)
df_char_it.equals(df_char_sql)

False

In [13]:
print(df_char_sql.head())
#it looks like it may just be the extra index column

   index            name  height mass hair_color   skin_color eye_color  \
0      1  Luke Skywalker   172.0   77      blond         fair      blue   
1      2           C-3PO   167.0   75       None         gold    yellow   
2      3           R2-D2    96.0   32       None  white, blue       red   
3      4     Darth Vader   202.0  136       none        white    yellow   
4      5     Leia Organa   150.0   49      brown        light     brown   

  birth_year  gender homeworld species  
0      19BBY    male  Tatooine   Human  
1     112BBY    None  Tatooine   Droid  
2      33BBY    None     Naboo   Droid  
3    41.9BBY    male  Tatooine   Human  
4      19BBY  female  Alderaan   Human  


In [14]:
df_char_sql = df_char_sql >> drop(['index'])

In [15]:
df_char_sql.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species
0,Luke Skywalker,172.0,77,blond,fair,blue,19BBY,male,Tatooine,Human
1,C-3PO,167.0,75,None,gold,yellow,112BBY,None,Tatooine,Droid
2,R2-D2,96.0,32,None,"white, blue",red,33BBY,None,Naboo,Droid
3,Darth Vader,202.0,136,none,white,yellow,41.9BBY,male,Tatooine,Human
4,Leia Organa,150.0,49,brown,light,brown,19BBY,female,Alderaan,Human


In [16]:
#crap still not equal
df_char_usual.equals(df_char_sql)
df_char_it.equals(df_char_sql)
#maybe the datatypes?

False

In [17]:
df_char_usual.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species
0,Luke Skywalker,172.0,77,blond,fair,blue,19BBY,male,Tatooine,Human
1,C-3PO,167.0,75,NaN,gold,yellow,112BBY,NaN,Tatooine,Droid
2,R2-D2,96.0,32,NaN,"white, blue",red,33BBY,NaN,Naboo,Droid
3,Darth Vader,202.0,136,none,white,yellow,41.9BBY,male,Tatooine,Human
4,Leia Organa,150.0,49,brown,light,brown,19BBY,female,Alderaan,Human


In [18]:
#no, that's not it either
print(df_char_usual.dtypes)
print(df_char_sql.dtypes)

name           object
height        float64
mass           object
hair_color     object
skin_color     object
eye_color      object
birth_year     object
gender         object
homeworld      object
species        object
dtype: object
name           object
height        float64
mass           object
hair_color     object
skin_color     object
eye_color      object
birth_year     object
gender         object
homeworld      object
species        object
dtype: object


In [23]:
#well that's not the same
#it looks like maybe the sql one had a duplicate row for each?
print('The shape of the data imported the usual way is :', df_char_usual.shape)
print('The shape of the data imported the database way is :',df_char_sql.shape)

The shape of the data imported the usual way is : (87, 10)
The shape of the data imported the database way is : (174, 10)


In [20]:
#I think it would be helpful to close and dispose of the engine and then try again
#because I ran the code a few times so I think it kept re-appending
csv_database.dispose()
os.remove('csv_database.db')

In [21]:
#the above seems to have 
#closed and dispoed of the engine
print(pd.read_sql_query("SELECT * FROM 'table'", csv_database))

OperationalError: (sqlite3.OperationalError) no such table: table
[SQL: SELECT * FROM 'table']
(Background on this error at: http://sqlalche.me/e/e3q8)

In [22]:
#rebuilding the engine and redoing everything
#and then checking if everything is equal
csv_database = create_engine('sqlite:///csv_database.db')

chunksize = 100000
i = 0
j = 1
for df in pd.read_csv(file, chunksize = chunksize, iterator = True):
      df.index += j
      i+=1
      df.to_sql('table', csv_database, if_exists = 'append')
      j = df.index[-1] + 1
    
df_char_sql1 = pd.read_sql_query("SELECT * FROM 'table'", csv_database)

df_char_sql1 = df_char_sql1 >> drop(['index'])

df_char_usual.equals(df_char_sql1)
df_char_it.equals(df_char_sql1)
#YAYAYAYAY

True